In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import beta
import seaborn as sns
import geopandas as gpd
from scipy.stats import gamma, lognorm, norm
import random
import os
import rasterio
from scipy.ndimage import uniform_filter
from rasterio.features import geometry_mask

import pandas as pd
import numpy as np

def sample_categorical_census(df: pd.DataFrame, category_col: str, value_col: str, ignore_categories: list):
    """
    Sample from a categorical distribution.
    """

    # Filter ignored categories
    if ignore_categories:
        df = df[~df[category_col].isin(ignore_categories)]

    totals = df.groupby(category_col)[value_col].sum()
    probs = totals / totals.sum()

    samples = np.random.choice(
        probs.index.to_numpy(),
        size=1,
        p=probs.to_numpy()
    )

    return samples[0]


# Grid

### Water area

In [ ]:
gdf = gpd.read_file("data/watercourse_density/watercourse_density.shp")
length = gdf["length_km"]
pct_zero = (length == 0).sum() / len(length)
zero_sample = np.random.binomial(n=1, p=pct_zero, size=1)[0]

gdf_nonzero = gdf[gdf["length_km"] != 0].copy()
length_nonzero = gdf_nonzero["length_km"]
shape, loc, scale = gamma.fit(length_nonzero)
length_sample = 0 if zero_sample == 1 else gamma.rvs(a=shape, loc=loc, scale=scale, size=1)[0]
print(length_sample*1000) # metres


### Distance to water 

In [ ]:
gdf = gpd.read_file("data/watercourse_distance/watercourse_distance.shp")
distance = gdf["distance_t"]
shape, loc, scale = gamma.fit(distance)

max_distance = distance.max()
epsilon = 0.01
scale_factor = max_distance / (length_sample + epsilon)

distance_sample = gamma.rvs(a=shape, loc=loc, scale=scale, size=10)
#if length_sample == 0:
#    distance_sample = max_distance - distance_sample
print(distance_sample) # metres



### Flood risk level

In [ ]:
'''risk_shapefile = [
    'data/flood_risk/rofsw_4bandPolygon/merged_rofsw_4bandPolygon.shp',
    'data/flood_risk/rofsw_4band_0_2m_depthPolygon/merged_rofsw_4band_0_2m_depthPolygon.shp',
    'data/flood_risk/rofsw_4band_0_3m_depthPolygon/merged_rofsw_4band_0_3m_depthPolygon.shp',
    'data/flood_risk/rofsw_4band_0_6m_depthPolygon/merged_rofsw_4band_0_6m_depthPolygon.shp',
    'data/flood_risk/rofsw_4band_0_9m_depthPolygon/merged_rofsw_4band_0_9m_depthPolygon.shp',
    'data/flood_risk/rofsw_4band_1_2m_depthPolygon/merged_rofsw_4band_1_2m_depthPolygon.shp'
]

gm_gdf = gpd.read_file("GM_shapefile/CAUTH_MAY_2025_EN_BSC.shp")
gm_area_km2 = gm_gdf.geometry.area.sum() / 1000000
print('Total area of Greater Manchester:')
print(str(gm_area_km2)+' km²')

# high > 3.3%
# medium 1-3.3% 
# low 0.1-1%
# very low < 0.1%

gdf = gpd.read_file('data/flood_risk/rofsw_4bandPolygon/merged_rofsw_4bandPolygon.shp')
gdf["area_m2"] = gdf.geometry.area
gdf["area_km2"] = gdf["area_m2"] / 1000000
total = 0
depth_risk_prob = {}
for depth_shapefile in risk_shapefile:
    depth_gdf = gpd.read_file(depth_shapefile)
    depth_gdf["area_m2"] = depth_gdf.geometry.area
    depth_gdf["area_km2"] = depth_gdf["area_m2"] / 1000000
    depth_total = 0
    risk_prob = {}
    for risk in ['Very low' , 'Low', 'Medium', 'High']:
        subset = depth_gdf[depth_gdf["risk_band"] == risk]
        total_area_km2 = subset["area_km2"].sum()
        depth_total += total_area_km2
        risk_prob[risk] = float(total_area_km2 / gm_area_km2)
    risk_prob['No Risk'] = float((gm_area_km2 - depth_total) / gm_area_km2)
    depth_risk_prob[depth_shapefile] = risk_prob
print(depth_risk_prob)

depths = list(depth_risk_prob.keys())
high_risk_probs = np.array([depth_risk_prob[d]['High'] for d in depths])
high_risk_probs = high_risk_probs / high_risk_probs.sum()
print(high_risk_probs)
depth_choice = np.random.choice(depths, p=high_risk_probs)
print(depth_choice)

risk_probs = depth_risk_prob[depth_choice]
risks = list(risk_probs.keys())
risk_prob_values = np.array([risk_probs[r] for r in risks])
risk_prob_values = risk_prob_values / risk_prob_values.sum()
print(risk_prob_values)
risk_choice = np.random.choice(risks, p=risk_prob_values)
print(risk_choice)'''



### Elevation

In [ ]:
'''
Exclude outer bound values (0), average windows, Gamma
'''

window_size = 7 # nxn

with rasterio.open("data/elevation.tif") as src:
    elevation = src.read(1)

valid_mask = elevation != 0
elev_filled = np.where(valid_mask, elevation, 0)  # set zeros to 0 for filtering

# Compute sum over windows
sum_window = uniform_filter(elev_filled.astype(float), size=window_size, mode='constant', cval=0)

# Compute count of valid pixels in each window
count_window = uniform_filter(valid_mask.astype(float), size=window_size, mode='constant', cval=0)

# Compute average, avoiding division by zero
avg_window = sum_window / np.maximum(count_window, 1)

averages = avg_window.flatten()
averages = averages[count_window.flatten() > 0]  # remove windows with all zeros


In [ ]:
averages_pos = averages[averages > 0]
shape, loc, scale = gamma.fit(averages_pos, floc=0)

In [ ]:
elevation_sample = gamma.rvs(a=shape, loc=loc, scale=scale, size=1)[0]

print(f"Total windows averaged: {len(averages)}")
print(elevation_sample)

### Impervious surface area

In [ ]:
'''
Only pixels within boundary averaged in windows, Gamma, scales sample based on Urban/rural
'''

window_size = 7  # nxn window

gdf = gpd.read_file("GM_shapefile/CAUTH_MAY_2025_EN_BSC.shp")

with rasterio.open("data/elevation.tif") as src:
    elevation = src.read(1)
    transform = src.transform

# Create GM mask (True inside GM)
gm_mask = geometry_mask(geometries=gdf.geometry, transform=transform, invert=True, out_shape=elevation.shape)

# Mask outside GM only
elevation_gm = np.where(gm_mask, elevation, np.nan)

# Detect windows with ANY NaN
nan_mask = np.isnan(elevation_gm)
nan_count = uniform_filter(nan_mask.astype(float), size=window_size, mode="constant", cval=1)

# Compute sum (NaNs replaced only for summation) 
sum_window = uniform_filter(np.nan_to_num(elevation_gm), size=window_size, mode="constant", cval=0)

# Compute average only for fully valid windows 
valid_windows = nan_count == 0
avg_window = np.full(elevation.shape, np.nan)
avg_window[valid_windows] = sum_window[valid_windows] / (window_size ** 2)

averages = avg_window[~np.isnan(avg_window)]



In [ ]:
averages_pos = averages[averages > 0]
shape, loc, scale = gamma.fit(averages_pos, floc=0)

impervious_sample = gamma.rvs(a=shape, loc=loc, scale=scale, size=1)[0]

urban_factor = 1.15  
rural_factor = 0.85

factor = urban_factor if urban_sample else rural_factor
impervious_sample = impervious_sample * factor

print(f"Total windows averaged: {len(averages)}")
print(impervious_sample)

### Historic flood map

### Road network density

# Temp

### 1. URBAN RURAL

In [ ]:
'''
Bernoulli
'''

urban = pd.read_csv("data/urban_rural.csv")
counts = urban['Urban_rural_flag'].value_counts()
urban_probs = counts / counts.sum()
p_urban = urban_probs['Urban']
urban_sample = np.random.binomial(n=1, p=p_urban, size=1)[0]
print(urban_sample)

### 2. POPULATION DENSITY

In [ ]:
'''
Lognorm
Dependencies: Urban/rural
'''

popden = pd.read_csv("data/population_density.csv", dtype={"LAD2021": "string", "OA21CD": "string", "Total": "Int64"})
merged = popden.merge(urban, on="OA21CD", how="left")
popden_urbanrural = merged[['LAD2021','OA21CD','Total','Urban_rural_flag']]

flag = 'urban' if urban_sample == 1 else 'rural'
popden_total  = popden_urbanrural[popden_urbanrural["Urban_rural_flag"].str.lower() == flag]["Total"]

log = np.log(popden_total)

mu, sigma = log.mean(), log.std()

popden_sample = np.random.lognormal(mean=mu, sigma=sigma, size=1)[0]
print(popden_sample)

### 3. DISABILITY RATE

In [ ]:
'''
Beta 
'''

disability = pd.read_csv("data/disabled.csv")
total_disability = disability.groupby('Disability (3 categories)')['Observation'].sum()
#disability_probs = total_disability / total_disability.sum()

alpha = total_disability['Disabled under the Equality Act'] + 1
beta = total_disability['Not disabled under the Equality Act'] + 1

disabled_sample = np.random.beta(alpha, beta, size=1)[0]
print(disabled_sample)


### 4. ENGLISH PROFICIENCY

In [ ]:
'''
Beta 
Dependencies: Urban/rural
'''

def map_proficiency(category):
    if category in [
        "Main language is English (English or Welsh in Wales)",
        "Main language is not English (English or Welsh in Wales): Can speak English very well or well"
    ]:
        return "Good English Proficiency"
    elif category == "Main language is not English (English or Welsh in Wales): Cannot speak English or cannot speak English well":
        return "Bad English Proficiency"
    else:
        return None

english = pd.read_csv("data/english_proficiency.csv")   
total_english = english.groupby('Proficiency in English language (4 categories)')['Observation'].sum()
english['Proficiency_Group'] = english['Proficiency in English language (4 categories)'].apply(map_proficiency)
grouped_english = english.groupby('Proficiency_Group')['Observation'].sum()
#english_probs = grouped_english / grouped_english.sum()

alpha = grouped_english['Good English Proficiency'] + 1
beta = grouped_english['Bad English Proficiency'] + 1

english_sample = np.random.beta(alpha, beta, size=1)[0]
print(english_sample)

### 7. MEAN PROPERTY VALUE

In [ ]:
property = pd.read_csv("data/property_value.csv")
property = property.dropna(subset=['price', 'property_type', 'duration'])
property = property[property['price'] > 0]

log_property = np.log(property['price'])
shape, loc, scale = skewnorm.fit(log_property)
sample_log = skewnorm.rvs(shape, loc=loc, scale=scale, size=1)
property_sample = np.exp(sample_log)[0]
print(property_sample)


### 9. BUILDING AGE

In [ ]:
building_age = pd.read_csv("data/property_age.csv")
age_columns = [
    "BP_PRE_1900","BP_1900_1918","BP_1919_1929","BP_1930_1939",
    "BP_1945_1954","BP_1955_1964","BP_1965_1972","BP_1973_1982",
    "BP_1983_1992","BP_1993_1999","BP_2000_2009","BP_2010_2015"
]
age_totals = building_age[age_columns].sum()
age_totals.index = [
    "Pre-1900","1900-1918","1919-1929","1930-1939","1945-1954","1955-1964",
    "1965-1972","1973-1982","1983-1992","1993-1999","2000-2009","2010-2015"
]
age_probs = age_totals / age_totals.sum()
age_categories = age_totals.index.tolist()
building_age_sample = np.random.choice(age_categories, size=1, p=age_probs)[0]
print(building_age_sample)



### 10. 24-Hour Precipitation and 13. Flood Depth (NOT DONE)

In [ ]:
precipitation = pd.read_csv("data/precipitation.csv")
rainfall = precipitation["rainfall"]
pct_zero = (rainfall == 0).sum() / len(rainfall)
zero_sample = np.random.binomial(n=1, p=pct_zero, size=1)[0]

precipitation_nonzero = precipitation[precipitation["rainfall"] != 0].copy()
rainfall_nonzero = precipitation["rainfall"]
max_prec = max(rainfall_nonzero)
shape, loc, scale = gamma.fit(rainfall_nonzero)
prec_sample = 0 if zero_sample == 1 else gamma.rvs(a=shape, loc=loc, scale=scale, size=1)[0]
print(prec_sample)

'''if prec_sample == 0:
    percentile = 0.0
else:
    percentile = gamma.cdf(prec_sample, a=shape, loc=loc, scale=scale)

if percentile < 0.97:
    flood_depth = 0.0
elif percentile < 0.99:
    flood_depth = 0.2
elif percentile < 0.999:
    flood_depth = 0.3
elif percentile < 0.9999:
    flood_depth = 0.6
elif percentile < 0.99999:
    flood_depth = 0.9
else:
    flood_depth = 1.2

print(gamma.ppf(0.97, a=shape, loc=loc, scale=scale))
print(gamma.ppf(0.99, a=shape, loc=loc, scale=scale))
print(gamma.ppf(0.999, a=shape, loc=loc, scale=scale))
print(gamma.ppf(0.9999, a=shape, loc=loc, scale=scale))
print(gamma.ppf(0.99999, a=shape, loc=loc, scale=scale))
print(flood_depth)  # in metres'''



### 11. Emergency Response Times

In [ ]:
response = pd.read_csv("data/response_times.csv")
def hhmmss_to_hours(s):
    h, m, sec = map(int, s.split(":"))
    return h + m/60 + sec/3600

c2_mean = response["C2_mean"].apply(hhmmss_to_hours)
c3_mean = response["C3_mean"].apply(hhmmss_to_hours)
c2_count = response['C2_count'].str.replace(',', '').astype(int).sum()
c3_count = response['C3_count'].str.replace(',', '').astype(int).sum()
c2_prob = c2_count / (c2_count + c3_count)
c3_prob = c3_count / (c2_count + c3_count)
response_category = np.random.choice(['C2', 'C3'], size=1, p=[c2_prob, c3_prob])[0]
if response_category == 'C2':
    shape, loc, scale = lognorm.fit(c2_mean, floc=0)
else:
    shape, loc, scale = lognorm.fit(c3_mean, floc=0)
response_sample = lognorm.rvs(shape, loc=loc, scale=scale, size=1)[0]
#print(response_sample) # hours

popden_mean = popden_total.mean()
if popden_sample < popden_mean: # scale based on average population density
    popden_factor = 1 - 0.5 * ((popden_mean - popden_sample) / popden_mean)  # reduce scale
else:
    popden_factor = 1 + 0.5 * ((popden_sample - popden_mean) / popden_mean)
prec_factor = np.exp(prec_sample / 50) # small rain = minimal effect, heavy rain = large effect
print(popden_factor, prec_factor)
adjusted_response = response_sample * popden_factor * prec_factor
print(adjusted_response)


### 14. Day of the week

In [ ]:
days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
day_sample = random.choice(days)
print(day_sample)

### 15. Season

In [ ]:
season = ['Winter', 'Spring', 'Summer', 'Autumn']
season_sample = random.choice(season)
print(season_sample)

### 16. Holiday binary

In [ ]:
'''
Min 28 days off annually
'''
p = 28 / 365
holiday_binary_sample = 1 if random.random() < p else 0
print(holiday_binary_sample)

### 18. Depth-damage curve (ON HOLD)

### 19. Groundwater level (WORK IN PREC)

In [ ]:
'''
Choose random groundwater station, Gamma
'''

# mAOD
# add prec dependency, modify ground water level based on previous prec_sample

groundwater_files = [f for f in os.listdir("data/groundwater_level") if f.lower().endswith(".csv")]
gw_file = random.choice(groundwater_files)

path = os.path.join("data/groundwater_level", gw_file)
df = pd.read_csv(path)

values = pd.to_numeric(df["value"], errors="coerce").dropna()
a, loc, scale = skewnorm.fit(values)
gw_sample = skewnorm.rvs(a, loc, scale, size=1)[0]

print(f"Selected file: {gw_file}")
print(gw_sample)



### 20. River flow (WORK IN PREC)

In [ ]:
'''
Random choose river station, Gamma
'''

# m3/s
# add prec dependency, modify ground water level based on previous prec_sample

river_flow_files = [f for f in os.listdir("data/river_flow") if f.lower().endswith(".csv")]
rf_file = random.choice(river_flow_files)

path = os.path.join("data/river_flow", rf_file)
df = pd.read_csv(path)

values = pd.to_numeric(df["value"], errors="coerce").dropna()
shape, loc, scale = gamma.fit(values, floc=0)
rf_sample = gamma.rvs(a=shape, loc=loc, scale=scale, size=1)[0]

print(f"Selected file: {rf_file}")
print(rf_sample)


### 21. River level (WORK IN PREC)

In [ ]:
'''
Using same river station as river flow, Gamma
'''
# m
# add prec dependency, modify ground water level based on previous prec_sample

file_prefix = rf_file.split('-')[0]
river_level_files = [f for f in os.listdir("data/river_level") if f.lower().endswith(".csv")]
for f in river_level_files:
    if file_prefix in f:
        rl_file = f
        break
    
path = os.path.join("data/river_level", rl_file)
df = pd.read_csv(path)

values = pd.to_numeric(df["value"], errors="coerce").dropna()
shape, loc, scale = gamma.fit(values, floc=0)
rl_sample = gamma.rvs(a=shape, loc=loc, scale=scale, size=1)[0]

print(f"Selected file: {rl_file}")
print(rl_sample)


### 22. Soil moisture saturation (WORK IN PREC and TEMP)

In [ ]:
'''
Random choose pixel, sample from normal distribution of time-series values for the pixel
'''

tiff_files = sorted(glob.glob("data/soil_moisture/*.tif"))

stack = []
profile = None

for f in tiff_files:
    with rasterio.open(f) as src:
        if profile is None:
            profile = src.profile
        data = src.read(1).astype(np.float32)
        nodata = src.nodata
        if nodata is not None:
            data[data == nodata] = np.nan
        stack.append(data)

# Shape: (time, rows, cols)
stack = np.stack(stack, axis=0)

time, rows, cols = stack.shape

i = random.randint(0, rows-1)
j = random.randint(0, cols-1)

values = stack[:, i, j]
values = values[~np.isnan(values)]  # remove nodata

# Normal distribution
mu, sigma = norm.fit(values)


In [ ]:
soil_sample = norm.rvs(mu, sigma, size=1)[0]

print(f"Random pixel chosen: ({i},{j})")
print(soil_sample)

### 23. Impervious surface area

In [ ]:
'''
Only pixels within boundary averaged in windows, Gamma, scales sample based on Urban/rural
'''

window_size = 7  # nxn window

gdf = gpd.read_file("GM_shapefile/CAUTH_MAY_2025_EN_BSC.shp")

with rasterio.open("data/elevation.tif") as src:
    elevation = src.read(1)
    transform = src.transform

# Create GM mask (True inside GM)
gm_mask = geometry_mask(geometries=gdf.geometry, transform=transform, invert=True, out_shape=elevation.shape)

# Mask outside GM only
elevation_gm = np.where(gm_mask, elevation, np.nan)

# Detect windows with ANY NaN
nan_mask = np.isnan(elevation_gm)
nan_count = uniform_filter(nan_mask.astype(float), size=window_size, mode="constant", cval=1)

# Compute sum (NaNs replaced only for summation) 
sum_window = uniform_filter(np.nan_to_num(elevation_gm), size=window_size, mode="constant", cval=0)

# Compute average only for fully valid windows 
valid_windows = nan_count == 0
avg_window = np.full(elevation.shape, np.nan)
avg_window[valid_windows] = sum_window[valid_windows] / (window_size ** 2)

averages = avg_window[~np.isnan(avg_window)]



In [ ]:
averages_pos = averages[averages > 0]
shape, loc, scale = gamma.fit(averages_pos, floc=0)

impervious_sample = gamma.rvs(a=shape, loc=loc, scale=scale, size=1)[0]

urban_factor = 1.15  
rural_factor = 0.85

factor = urban_factor if urban_sample else rural_factor
impervious_sample = impervious_sample * factor

print(f"Total windows averaged: {len(averages)}")
print(impervious_sample)

### 24. General health

In [ ]:
df = pd.read_csv('data/general_health.csv')
gen_health_sample = sample_categorical_census(df, 
                                                'General health (4 categories)', 
                                                'Observation', 
                                                ['Does not apply'])
print(gen_health_sample)

### 26. Age

In [ ]:
df = pd.read_csv('data/age.csv')
age_sample = sample_categorical_census(df,
                                       'Age (6 categories)',
                                       'Observation',
                                       [])
print(age_sample)

### 27. Elderly

In [ ]:
elderly_sample = 1 if age_sample == 'Aged 65 years and over' else 0
print(elderly_sample)

### 28. Children

In [ ]:
child_sample = 1 if age_sample == 'Aged 15 years and under' else 0
print(child_sample)

### 29. Employment history

In [ ]:
'''
Does not apply: Either child or in employment
'''

employ = pd.read_csv('data/employment-age.csv')
filtered_employ = employ[employ['Age (6 categories)'] == age_sample]

employ_sample = sample_categorical_census(filtered_employ,
                                       'Employment history (4 categories)',
                                       'Observation',
                                       [])

print(employ_sample)

### 30. Highest level of qualification

In [ ]:
'''
Does not apply is only for <15, 
'''
qual = pd.read_csv('data/qualification-age.csv')
filtered_qual = qual[qual['Age (6 categories)'] == age_sample]

qual_sample = sample_categorical_census(filtered_qual,
                                       'Highest level of qualification (7 categories)',
                                       'Observation',
                                       [])
print(qual_sample)

### 31. Lifestage of household reference person

In [ ]:
lifestage = pd.read_csv('data/lifestage_hrp_age.csv')
filtered_lifestage = lifestage[lifestage['Age (6 categories)'] == age_sample]
lifestage_sample = sample_categorical_census(filtered_lifestage,
                                       'Lifestage of Household Reference Person(13 categories)',
                                       'Observation',
                                       ['Does not apply'])
print(lifestage_sample)

### 32. Accomodation type

In [ ]:
acco_type = pd.read_csv('data/accomodation_type.csv')
acco_type_sample = sample_categorical_census(acco_type,
                                       'Accommodation type (5 categories)',
                                       'Observation',
                                       [])
print(acco_type_sample)

### 33. Vehicle

In [ ]:
vehicle = pd.read_csv('data/vehicle.csv')
vehicle_sample = sample_categorical_census(vehicle,
                                       'Car or van availability (3 categories)',
                                       'Observation',
                                       ['Does not apply'])
print(vehicle_sample)

### 34. Second address

In [ ]:
second_add = pd.read_csv('data/second_address.csv')
second_add_sample = sample_categorical_census(second_add,
                                       'Second address indicator (3 categories)',
                                       'Observation',
                                       [])
print(second_add_sample)

### 35. Household size

In [ ]:
house_size = pd.read_csv('data/household_size.csv')
house_size_sample = sample_categorical_census(house_size,
                                       'Household size (5 categories)',
                                       'Observation',
                                       ['0 people in household'])
print(house_size_sample)

### 36. Economic activity status

In [ ]:
'''
Does not apply is only for <15, 
'''
eas = pd.read_csv('data/nssec_economic_age.csv')
filtered_eas = eas[eas['Age (6 categories)'] == age_sample]

eas_sample = sample_categorical_census(filtered_eas,
                                       'Economic activity status (4 categories)',
                                       'Observation',
                                       [])
print(eas_sample)

### 25. Ns-SeC

In [ ]:
nssec = pd.read_csv('data/nssec_economic_age.csv')
filtered_nssec = nssec[nssec['Age (6 categories)'] == age_sample]
filtered_nssec = filtered_nssec[filtered_nssec['Economic activity status (4 categories)'] == eas_sample]
nssec_sample = sample_categorical_census(filtered_nssec, 
                                         'National Statistics Socio-economic Classification (NS-SeC) (10 categories)', 
                                         'Observation',
                                         [])
print(nssec_sample)

### 5. MEAN INCOME (REPLACE WEIGHTS WITH DATA-DRIVEN CHOICES)

In [ ]:
'''
Log skewnorm 
'''

from scipy.stats import skewnorm, lognorm

income = pd.read_csv("data/mean_income.csv")
income['Total annual income (£)'] = (
    income['Total annual income (£)']
    .str.strip()        
    .str.replace(',', '')      
    .astype(float)       
)

log_income = np.log(income['Total annual income (£)'])
shape, loc, scale = skewnorm.fit(log_income)
sample_log = skewnorm.rvs(shape, loc=loc, scale=scale, size=1)
income_sample = np.exp(sample_log)[0]

NSSEC = {
    "L1, L2 and L3: Higher managerial, administrative and professional occupations": 1.90,
    "L4, L5 and L6: Lower managerial, administrative and professional occupations": 1.35,
    "L7: Intermediate occupations": 1.00,
    "L8 and L9: Small employers and own account workers": 1.10,
    "L10 and L11: Lower supervisory and technical occupations": 0.90,
    "L12: Semi-routine occupations": 0.75,
    "L13: Routine occupations": 0.65,
    "L14.1 and L14.2: Never worked and long-term unemployed": 0.40,
    "L15: Full-time students": 0.35,
    "Does not apply": 0.00
}

income_sample *= NSSEC[nssec_sample]
print(income_sample)

### 6. LOW-INCOME FRACTION (NOT DONE)

In [ ]:

median = income['Total annual income (£)'].median()
low_income_threshold = 0.6 * median
print(low_income_threshold)

variance_log_skewnorm = skewnorm.var(shape, loc=loc, scale=scale)

###### PART THAT DOESNT SEEM RIGHT
meanlog = np.log(income_sample)
std = np.sqrt(variance_log_skewnorm)
print(std)
prob_low_income = lognorm.cdf(low_income_threshold, s=std, scale=np.exp(meanlog))
print(prob_low_income)


### 37. No. adults employed in household

In [ ]:
num_adults = pd.read_csv('data/household_employed_size.csv')
filtered_num_adults = num_adults[num_adults['Household size (5 categories)'] == house_size_sample]

num_adults_sample = sample_categorical_census(filtered_num_adults,
                                       'Number of adults in employment in household (5 categories)',
                                       'Observation',
                                       ['Does not apply'])
print(num_adults_sample)

### 38. No. disabled people household 

In [ ]:
num_disable = pd.read_csv('data/household_disabled_size.csv')
filtered_num_disable = num_disable[num_disable['Household size (5 categories)'] == house_size_sample]

num_disable_sample = sample_categorical_census(filtered_num_disable,
                                       'Number of disabled people in household (4 categories)',
                                       'Observation',
                                       ['Does not apply'])
print(num_disable_sample)

### 39. No. long-term health in household

In [ ]:
num_long = pd.read_csv('data/household_long-term_size.csv')
filtered_num_long = num_long[num_long['Household size (5 categories)'] == house_size_sample]

num_long_sample = sample_categorical_census(filtered_num_long,
                                       'Number of people in household with a long-term heath condition but are not disabled (4 categories)',
                                       'Observation',
                                       ['Does not apply'])
print(num_long_sample)

### 40. Deprived in education

In [ ]:
dep_edu = pd.read_csv('data/deprived_education+deps.csv')
filtered_dep_edu = dep_edu[dep_edu['Highest level of qualification (7 categories)'] == qual_sample]

dep_edu_sample = sample_categorical_census(filtered_dep_edu,
                                       'Household deprived in the education dimension (3 categories)',
                                       'Observation',
                                       ['Does not apply'])
print(dep_edu_sample)

### 41. Deprived in employment

In [ ]:
dep_employ = pd.read_csv('data/deprived_employment+deps.csv')
filtered_dep_employ = dep_employ[dep_employ['Employment history (4 categories)'] == employ_sample]
filtered_dep_employ = filtered_dep_employ[filtered_dep_employ['National Statistics Socio-economic Classification (NS-SeC) (10 categories)'] == nssec_sample]
dep_employ_sample = sample_categorical_census(filtered_dep_employ,
                                       'Household deprived in the employment dimension (3 categories)',
                                       'Observation',
                                       [])
print(dep_employ_sample)

### 42. Deprived in health and disability

In [ ]:
dep_health = pd.read_csv('data/deprived_health+deps.csv')
filtered_dep_health = dep_health[dep_health['Number of people in household with a long-term heath condition but are not disabled (4 categories)'] == num_long_sample]
filtered_dep_health = filtered_dep_health[filtered_dep_health['Number of disabled people in household (4 categories)'] == num_disable_sample]
dep_health_sample = sample_categorical_census(filtered_dep_health,
                                       'Household deprived in the health and disability dimension (3 categories)',
                                       'Observation',
                                       [])
print(dep_health_sample)

### 43. No. of people per room in household 

In [ ]:
num_people = pd.read_csv('data/people_per_room_hsize.csv')
filtered_num_people = num_people[num_people['Household size (5 categories)'] == house_size_sample]
num_people_sample = sample_categorical_census(filtered_num_people,
                                       'Number of people per room in household (5 categories)',
                                       'Observation',
                                       ['Does not apply'])
print(num_people_sample)

### 44. Occupancy rating for rooms

In [ ]:
occupancy = pd.read_csv('data/occupancy_rating_nopeopleper.csv')
filtered_occupancy = occupancy[occupancy['Number of people per room in household (5 categories)'] == num_people_sample]
num_occupancy = sample_categorical_census(filtered_occupancy,
                                       'Occupancy rating for rooms (5 categories)',
                                       'Observation',
                                       ['Does not apply'])
print(num_occupancy)

### 45. Deprived in housing

In [ ]:
dep_housing = pd.read_csv('data/deprived_housing+deps.csv')
filtered_dep_housing = dep_housing[dep_housing['Number of people per room in household (5 categories)'] == num_people_sample]
filtered_dep_housing = filtered_dep_housing[filtered_dep_housing['Occupancy rating for rooms (5 categories)'] == num_occupancy]
dep_housing_sample = sample_categorical_census(filtered_dep_housing,
                                       'Household deprived in the housing dimension (3 categories)',
                                       'Observation',
                                       ['Does not apply'])
print(dep_housing_sample)

### 46. Household deprivation

In [ ]:
# Smaller = less deprived
household_dep_sample = (dep_edu_sample == 'Household is deprived in the education dimension') + (dep_employ_sample == 'Household is deprived in the employment dimension') + (dep_health_sample == 'Household is deprived in the health and disability dimension') + (dep_housing_sample == 'Household is deprived in the housing dimension')
print(household_dep_sample)

### 47. Tenure of household

In [ ]:
tenure = pd.read_csv('data/tenure.csv')
tenure_sample = sample_categorical_census(tenure,
                                       'Tenure of household (7 categories)',
                                       'Observation',
                                       ['Does not apply'])
print(tenure_sample)

### 48. Household access to internet

In [ ]:
'''
linear regression

household_comp_sample

YEAR = 2026
time = np.array([2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020])
if household_comp_sample == 'One-person household: Aged 66 years and over':
    values = np.array([36, 40, 40, 49, 53, 61, 59, 73, 80])
elif household_comp_sample == 'One-person household: Other':
    values = np.array([76, 74, 81, 80, 87, 88, 91, 94, 95])
else:
    time = np.array([1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020])
    values = np.array([13, 25, 36, 42, 46, 49, 55, 57, 61, 65, 70, 73, 77, 80, 83, 84, 86, 89, 90, 90, 93, 96])

m, b = np.polyfit(time, values, 1)
val = m*YEAR+b
print(val)

all val > 100
'''
if lifestage_sample == 'Household reference person is aged 66 years or over: One-person household':
    prob = 0.85
else:
    prob = 0.98
internet_sample = np.random.binomial(n=1, p=prob)
print(internet_sample)


### 49. Household

In [ ]:
'''
Home ownership:
- wealth accumulation
- tenure security
Mortgage holders retain equity but face some financial exposure

Private renters:
- higher housing cost volatility
- lower security
Social renters:
- low-income
- benefit-dependent
- high-deprivation populations
'''
TENURE_RISK = {
    "Owned: Owns outright": 0.0,
    "Owned: Owns with a mortgage or loan or shared ownership": 0.1,
    "Private rented: Private landlord or letting agency": 0.6,
    "Private rented: Other private rented or lives rent free": 0.6,
    "Social rented: Rents from council or Local Authority": 0.8,
    "Social rented: Other social rented": 0.8,
}

'''
- Housing quality, space, and permanence decrease down the list
- Flats and temporary housing show higher overcrowding and energy risk
- Temporary structures are near-maximal risk 
'''
ACCO_RISK = {
    "Whole house or bungalow: Detached": 0.1,
    "Whole house or bungalow: Semi-detached": 0.1,
    "Whole house or bungalow: Terraced": 0.3,
    "Flat, maisonette or apartment": 0.5,
    "A caravan or other mobile or temporary structure": 0.9,
}

'''
Single-person households:
- income fragility
- social isolation risk
Two-person households:
- risk-sharing
Large households:
- crowding
- higher costs
- child dependency
'''
SIZE_RISK = {
    "1 person in household": 0.5,
    "2 people in household": 0.2,
    "3 people in household": 0.3,
    "4 or more people in household": 0.6,
}

'''
Internet access for weather warning services
'''
INTERNET_RISK = {
    1: 0.0,
    0: 0.6,
}

'''
Risk increases with less earners
'''
EMPLOYMENT_RISK = {
    "3 or more adults in employment in household": 0.0,
    "2 adults in employment in household": 0.2,
    "1 adult in employment in household": 0.5,
    "No adults in employment in household": 0.9,
}

'''
Deprivation in education, employment, health, housing
'''
DEPRIVATION_RISK = {
    0: 0.0,
    1: 0.25,
    2: 0.5,
    3: 0.75,
    4: 1.0,
}

'''
Older single households:
- health risks
- lower chance of internet access
- elderly vulnerable
Households with dependent children:
- cost pressure
- children vulnerable
Child-free working-age households are most resilient
'''
def lifestage_risk_map(x):
    if "66 years or over" in x and "One-person" in x:
        return 0.7
    if "Dependent children" in x:
        return 0.6
    if "Two or more person household: No dependent children" in x:
        return 0.3
    return 0.4

tenure_risk = TENURE_RISK[tenure_sample]
acco_risk = ACCO_RISK[acco_type_sample]
size_risk = SIZE_RISK[house_size_sample]
internet_risk = INTERNET_RISK[internet_sample]
employment_risk = EMPLOYMENT_RISK[num_adults_sample]
deprivation_risk = DEPRIVATION_RISK[household_dep_sample]
lifestage_risk = lifestage_risk_map(lifestage_sample)

'''
Deprivation: looks at 4 factors (higher weight)
Employment: Income stability
Employment: Wealth and security
Others: Secondary modifiers
'''
WEIGHTS = {
    "tenure_risk": 0.15,
    "acco_risk": 0.10,
    "size_risk": 0.10,
    "internet_risk": 0.10,
    "lifestage_risk": 0.10,
    "employment_risk": 0.20,
    "deprivation_risk": 0.25,
}
 
risk_score = (
    WEIGHTS["tenure_risk"]       * tenure_risk +
    WEIGHTS["acco_risk"]         * acco_risk +
    WEIGHTS["size_risk"]         * size_risk +
    WEIGHTS["internet_risk"]     * internet_risk +
    WEIGHTS["lifestage_risk"]    * lifestage_risk +
    WEIGHTS["employment_risk"]   * employment_risk +
    WEIGHTS["deprivation_risk"]  * deprivation_risk
)

print('Tenure -', tenure_sample)
print('Accomodation type -', acco_type_sample)
print('Household size -', house_size_sample)
print('Internet access -', internet_sample)
print('Adults employed -', num_adults_sample)
print('Household deprivation -', household_dep_sample)
print('Lifestage HRP -', lifestage_sample)

noise = np.random.normal(loc=0, scale=0.03) 
household_risk_score = np.clip(risk_score + noise, 0, 1)
print(household_risk_score)

### Road network density

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt

# Read the shapefile
gdf = gpd.read_file("data/road/MotorwayJunction.shp")

# View the first few rows (attribute table)
print(gdf.head())

# Visualise the first few features on a map
gdf.head().plot()
plt.show()

In [ ]:
gdf = gpd.read_file("data/road/RoadLink.shp")

# View the first few rows (attribute table)
print(gdf.head())

# Visualise the first few features on a map
gdf.head().plot()
plt.show()

In [ ]:
gdf = gpd.read_file("data/road/RoadNode.shp")

# View the first few rows (attribute table)
print(gdf.head())

# Visualise the first few features on a map
gdf.head().plot()
plt.show()

### 51. Ambulance handover delays

### 52. Hospital bed availability

### 53. Historic flood map